In [ ]:
# append path to find utils module in urbansim
import os 
import sys
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir) # to get path to utils module

In [ ]:
from sqlalchemy import create_engine
# from pysandag.database import get_connection_string
from database import get_connection_string
import pandas as pd
import folium

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

# Compare outputs

#### Get max run id

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
max_run_id = int(run_id_df.values)
print("\n Max Run id : {:,}".format(max_run_id))

### specify run_id either max run id or number

In [ ]:
run_id = max_run_id
run_id = 397

In [ ]:
print("\n   Run id : {:,}".format(run_id))
hs_change_sql = '''
    SELECT  o.parcel_id, jcpa, capacity_type, source, unit_change, year_simulation,
     COALESCE(cocpa_2016_name,cicpa_13_name,jurisdiction_2016_name) AS jcpa_name
      FROM urbansim.urbansim.urbansim_lite_output o
      JOIN [urbansim].[ref].[vi_parcel_xref] x
      ON o.parcel_id = x.parcel_id
     WHERE run_id =  %s
     ORDER BY year_simulation,parcel_id,jcpa,capacity_type,source
     '''
hs_change_sql = hs_change_sql % run_id
current_hs = pd.read_sql(hs_change_sql,mssql_engine)

### specify run_id to compare. max run id minus 1 or specify number

In [ ]:
run_id2 = run_id - 1
run_id2 = 390
# note run_id 133 matches 138

In [ ]:
print("\n   Previous run id : {:,}".format(run_id2))
hs_change_sql = '''
    SELECT o.parcel_id, jcpa, capacity_type, source, unit_change, year_simulation,
     COALESCE(cocpa_2016_name,cicpa_13_name,jurisdiction_2016_name) AS jcpa_name
      FROM urbansim.urbansim.urbansim_lite_output o
      JOIN [urbansim].[ref].[vi_parcel_xref] x
      ON o.parcel_id = x.parcel_id
     WHERE run_id =  %s
     ORDER BY year_simulation,parcel_id,jcpa,capacity_type,source
      '''
hs_change_sql = hs_change_sql % run_id2
previous_hs = pd.read_sql(hs_change_sql,mssql_engine)

In [ ]:
print("\nDo the simulations produce the same results?")
print(current_hs.equals(previous_hs))

## test one cpa

In [ ]:
#prev1447 = previous_hs.loc[previous_hs.jcpa==1447]
#cur1447 = current_hs.loc[current_hs.jcpa==1447]
#compare_1447 = pd.merge(prev1447,cur1447,on=['parcel_id','capacity_type','source','year_simulation','jcpa_name'])
#compare_1447['diff'] = compare_1447['unit_change_x'] - compare_1447['unit_change_y']
#compare_1447.loc[compare_1447['diff']!=0]

## data for plotting

In [ ]:
current = pd.DataFrame({'hs_sum': current_hs.groupby(['jcpa','year_simulation','jcpa_name']).\
                            unit_change.sum()}).reset_index()

In [ ]:
previous = pd.DataFrame({'hs_sum': previous_hs.groupby(['jcpa','year_simulation','jcpa_name']).\
                            unit_change.sum()}).reset_index()

In [ ]:
#current.loc[current.jcpa==1447]

In [ ]:
compare_df = current.merge(previous, on=['jcpa','jcpa_name','year_simulation'], how='outer',suffixes=['_current','_previous'])

In [ ]:
compare_df.loc[compare_df.jcpa==1447]

In [ ]:
compare_df.set_index('jcpa',inplace=True)
compare_df.set_index(['year_simulation'],append=True,inplace=True)
compare_df.set_index(['jcpa_name'],append=True,inplace=True)

In [ ]:
#compare_df.head()

In [ ]:
# fill in zero for years w no change

In [ ]:
idx = range(2017,2051)
compare_df = compare_df.unstack(['jcpa','jcpa_name'])
compare_df = compare_df.reindex(idx, fill_value=0)
compare_df.fillna(0,inplace=True)
compare_df = compare_df.stack(['jcpa','jcpa_name'])
compare_df.reset_index(inplace=True)

In [ ]:
# cumulative sum
compare_df['hs_current'] = compare_df.groupby(compare_df.jcpa)['hs_sum_current'].cumsum()
compare_df['hs_previous'] = compare_df.groupby(compare_df.jcpa)['hs_sum_previous'].cumsum()

In [ ]:
# c2 =  compare_df.loc[compare_df.jcpa.isin([1,2,3])][['jcpa','jcpa_name','year_simulation','hs_current','hs_previous']].copy()
c2 =  compare_df[['jcpa','jcpa_name','year_simulation','hs_current','hs_previous']].copy()

In [ ]:
#c2.head()


In [ ]:
#c2.loc[c2.jcpa==1447]

In [ ]:
dirname = os.path.join(os.getcwd(),'plots/run' + str(run_id) + '_' + str(run_id2))
try:
    os.stat(dirname)
except:
    os.mkdir(dirname)  
print(dirname)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
pdfplotname = dirname +  '//' + 'compare_runs_'  + str(run_id) + '_' + str(run_id2) + '.pdf'
# pp = PdfPages(pdfplotname)
pp = PdfPages(pdfplotname)

for jcpa in set(c2['jcpa']): 
    plotjcpa = c2[c2['jcpa']==jcpa].copy()
    jcpa_name = plotjcpa['jcpa_name'].iloc[0]
    jcpa_name_ = jcpa_name.replace(":", "_")
    plotjcpa.drop(['jcpa','jcpa_name'], axis=1,inplace=True)
    plotjcpa.plot(x='year_simulation',title ='%s jcpa=%d'% (jcpa_name,jcpa))
    plotname = dirname + '//' + 'compare_runs_' + str(jcpa) + '_' + jcpa_name_ + '.png'
    
    pp.savefig()
    plt.show()
    plt.clf()
    # pp.savefig(pdfplotname, dpi = 300, transparent = True)
pp.close()   

### testing to find duplicated parcel ids

In [ ]:
ids = current_hs['parcel_id']

In [ ]:
current_hs[ids.isin(ids[ids.duplicated()])].sort_values(by="parcel_id")

## Number of Parcels

In [ ]:
parcel_change_sql = '''
    Select count(o.parcel_id) as parcels_modified, jcpa, year_simulation
    FROM urbansim.urbansim.urbansim_lite_output o
    JOIN [urbansim].[ref].[vi_parcel_xref] x
    ON o.parcel_id = x.parcel_id
    WHERE run_id =  %s
    group by year_simulation, jcpa
    order BY year_simulation, jcpa
    '''
parcel_change_sql = parcel_change_sql % run_id2
previous_parcels = pd.read_sql(parcel_change_sql,mssql_engine)

In [ ]:
parcel_change_sql = '''
    Select count(o.parcel_id) as parcels_modified, jcpa, year_simulation
    FROM urbansim.urbansim.urbansim_lite_output o
    JOIN [urbansim].[ref].[vi_parcel_xref] x
    ON o.parcel_id = x.parcel_id
    WHERE run_id =  %s
    group by year_simulation, jcpa
    order BY year_simulation, jcpa
    '''
parcel_change_sql = parcel_change_sql % run_id
current_parcels = pd.read_sql(parcel_change_sql,mssql_engine)

In [ ]:
compare_parcels_df = current_parcels.merge(previous_parcels, on=['jcpa','year_simulation'],suffixes=['_current','_previous'])
#compare_parcels_df.set_index('jcpa',inplace=True)
#compare_parcels_df.set_index(['year_simulation'],append=True,inplace=True)
#compare_parcels_df

In [ ]:
pdfplotname = dirname +  '//' + 'compare_parcels_'  + str(run_id) + '_' + str(run_id2) + '.pdf'
# pp = PdfPages(pdfplotname)
pp = PdfPages(pdfplotname)

for jcpa in set(compare_parcels_df['jcpa']): 
    plotjcpa = compare_parcels_df[compare_parcels_df['jcpa']==jcpa].copy()
    plotjcpa.drop(['jcpa'], axis=1,inplace=True)
    plotjcpa.plot(x='year_simulation',title ='jcpa=%d'% (jcpa))
    plotname = dirname + '//' + 'compare_runs_' + str(jcpa) + '.png'
    
    pp.savefig()
    plt.show()
    plt.clf()
    # pp.savefig(pdfplotname, dpi = 300, transparent = True)
pp.close()   

# Map it

In [ ]:
m = folium.Map(location=[32.7157,  -117.1611])
m